In [4]:
%run "../00_project_config.ipynb"

import pathlib as pl
import pandas as pd

import re

RE_XFOLD_COV_DEC = re.compile("\d+\.\d+")
RE_XFOLD_COV = re.compile("\d+")

def norm_cov_value(value):
    
    try:
        number = float(value)
    except ValueError:
        has_int = RE_XFOLD_COV.search(value) is not None
        has_dec = RE_XFOLD_COV_DEC.search(value) is not None
        if "/" in value:
            v1, v2 = value.split("/")
            number = float(v1) + float(v2)
        elif has_dec:
            number = 0
            for xfold_cov in RE_XFOLD_COV_DEC.findall(value):
                number += float(xfold_cov)
        elif has_int:
            number = 0
            for xfold_cov in RE_XFOLD_COV.findall(value):
                number += float(xfold_cov)
        else:
            print(value)
            raise
    return number


cov_data_master_file = PROJECT_BASE.joinpath(
    "annotations", "projectmng", "master_data_cov.tsv"
)

cov_data_master = pd.read_csv(cov_data_master_file, sep="\t", comment="#", header=0)

cov_data_master["norm_hifi_cov"] = cov_data_master["hifi_cov"].apply(norm_cov_value)
cov_data_master["norm_ont_cov"] = cov_data_master["ont_cov"].apply(norm_cov_value)
cov_data_master["norm_ontul_cov"] = cov_data_master["ont_ul_cov"].apply(norm_cov_value)

sort_out = [
    "sample",
    "hifi_cov",
    "norm_hifi_cov",
    "ont_cov",
    "norm_ont_cov",
    "ont_ul_cov",
    "norm_ontul_cov"
]

norm_cov_out = PROJECT_BASE.joinpath(
    "annotations", "projectmng", "master_data_cov.norm.tsv"
)

with open(norm_cov_out, "w") as table:
    _ = table.write(f"# {TODAY}\n")
    cov_data_master[sort_out].to_csv(table, sep="\t", header=True, index=False)